In [2]:
# -*- coding: utf-8 -*-
"""
AI_Personal_Shopping_Assistant.py

Automatically generated.

Personal Shopping Assistant that helps users find products, compare prices, and notify about deals.
"""

import uuid
import yaml
import time
import re
from julep import Client

In [3]:
# Define UUIDs for tasks
AGENT_UUID = uuid.uuid4()
PRODUCT_SEARCH_TASK_UUID = uuid.uuid4()
DEAL_NOTIFICATION_TASK_UUID = uuid.uuid4()

# Initialize the Julep client
api_key = ""  # Replace with your actual Julep API key
client = Client(api_key=api_key, environment="dev")

In [4]:
# Create or update the AI agent
agent = client.agents.create_or_update(
    agent_id=AGENT_UUID,
    name="Personal Shopping Assistant",
    about="An AI assistant for helping users find products, compare prices, and notify about deals.",
    model="gpt-4o",
)

In [5]:
# Define the product search task
product_search_task_def = yaml.safe_load("""
name: Product Search

input_schema:
  type: object
  properties:
    search_query:
      type: string

main:
- prompt:
  - role: system
    content: >-
      You are a product search assistant. Search for products based on the following query:
      {{inputs[0].search_query}}

      Return a list of product names and their links.
  unwrap: true

- evaluate:
    products: _

- return:
    products: _
""")

product_search_task = client.tasks.create_or_update(
    task_id=PRODUCT_SEARCH_TASK_UUID,
    agent_id=AGENT_UUID,
    **product_search_task_def
)

In [6]:
# Define the deal notification task
deal_notification_task_def = yaml.safe_load("""
name: Deal Notification

input_schema:
  type: object
  properties:
    user_email:
      type: string
    deal_details:
      type: string

main:
- prompt:
  - role: system
    content: >-
      You are a notification assistant. Notify the user about the following deal:
      User Email: {{inputs[0].user_email}}
      Deal Details: {{inputs[0].deal_details}}

      Send a notification email to the user.
  unwrap: true

- evaluate:
    notification_status: "sent"  # Simulating a successful notification

- return:
    notification_status: _
""")

deal_notification_task = client.tasks.create_or_update(
    task_id=DEAL_NOTIFICATION_TASK_UUID,
    agent_id=AGENT_UUID,
    **deal_notification_task_def
)

In [7]:
def search_products(search_query):
    execution = client.executions.create(
        task_id=PRODUCT_SEARCH_TASK_UUID,
        input={
            "search_query": search_query
        }
    )
    time.sleep(2)
    result = client.executions.get(execution.id)
    output = client.executions.transitions.list(execution_id=result.id).items[0].output
    return output

In [8]:
def compare_prices(product_links):
    prices = {}

    products_list = re.findall(r'\*\*(.*?)\*\*', product_links)
    links_list = re.findall(r'\[(.*?)\]\((.*?)\)', product_links)

    for idx, product_name in enumerate(products_list):
        link = links_list[idx][1] if idx < len(links_list) else ""
        price = 200 + idx * 20  ]
        prices[product_name] = {"link": link, "price": price}

    if prices:
        best_product = min(prices.items(), key=lambda x: x[1]['price'])
        return f"{best_product[0]} is available for ${best_product[1]['price']} (Link: {best_product[1]['link']})"
    else:
        return "No valid products found for price comparison."

In [9]:
def notify_deal(user_email, deal_details):
    execution = client.executions.create(
        task_id=DEAL_NOTIFICATION_TASK_UUID,
        input={
            "user_email": user_email,
            "deal_details": deal_details
        }
    )
    time.sleep(2)
    result = client.executions.get(execution.id)
    return client.executions.transitions.list(execution_id=result.id).items[0].output

In [16]:
def display_results(product_links, best_deal, notification_status):
    print("\n" + "=" * 40)
    print("📦 Products Found:")
    print("=" * 40)
    if product_links:
        print("\n".join([f"- {product}" for product in product_links.splitlines()]))
    else:
        print("No products found.")

    print("\n" + "=" * 40)
    print("💸 Best Deal:")
    print("=" * 40)
    print(best_deal)

    # print("\n" + "=" * 40)
    # print("📧 Notification Status:")
    # print("=" * 40)
    # print("\n" + notification_status)

print("🚀 Demonstrating AI-Powered Personal Shopping Assistant:")

user_email = "user@example.com"
search_query = "smartphone under 300 dollars"
product_links = search_products(search_query)

if product_links:
    best_deal = compare_prices(product_links.get('products', {}).get('products', ""))
    notification_status = notify_deal(user_email, best_deal)
    display_results(product_links.get('products', {}).get('products', ""), best_deal, notification_status)
else:
    print("\nNo products found for the given search query.")


🚀 Demonstrating AI-Powered Personal Shopping Assistant:

📦 Products Found:
- Here is a list of smartphones under $300 along with their links:
- 
- 1. **Samsung Galaxy A14 5G**
-    - [Samsung Galaxy A14 5G on Amazon](https://www.amazon.com/Samsung-Galaxy-A14-5G/dp/B0BVZ9Z8B2)
- 
- 2. **Moto G Power (2023)**
-    - [Moto G Power 2023 on Amazon](https://www.amazon.com/Moto-G-Power-2023/dp/B0B8B8YRQH)
- 
- 3. **Nokia G20**
-    - [Nokia G20 on Amazon](https://www.amazon.com/Nokia-G20/dp/B094Y3H4Y9)
- 
- 4. **TCL 20 SE**
-    - [TCL 20 SE on Amazon](https://www.amazon.com/TCL-20-SE/dp/B08Z76QZQZ)
- 
- 5. **Samsung Galaxy A03s**
-    - [Samsung Galaxy A03s on Amazon](https://www.amazon.com/Samsung-Galaxy-A03s/dp/B09C1B3VRZ)
- 
- Please note that prices and availability may vary, so it's a good idea to check the links for the most current information.

💸 Best Deal:
Samsung Galaxy A14 5G is available for $200 (Link: https://www.amazon.com/Samsung-Galaxy-A14-5G/dp/B0BVZ9Z8B2)
